<a href="https://colab.research.google.com/github/heroza/sia-smote/blob/main/sia%20smote%20on%20cifar2%20exp%2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
no_exp = 23

exp_name="siamese"
dataset = 'cifar2'
dataset_name="128px_raw"
IMAGE_W = 32
IMAGE_H = 32
IMAGE_C = 3
num_classes = 2
epochs_siamese = 100
epochs_classifier = 20
num_folds = 5
isResnet = True

import myfunction as fu
from typing_extensions import Counter
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE

#LOAD DATASET
path = "/home/rh22708/data/"
print('loading dataset')
# (X_train, y_train), (X_test, y_test) = fu.load_breakhis(path, dataset, dataset_name, IMAGE_H, IMAGE_W,IMAGE_C)
# (X_train, y_train), (X_test, y_test) = fu.load_isic2018(path, dataset, dataset_name, IMAGE_H, IMAGE_W,IMAGE_C)
(X_train, y_train), (X_test, y_test) = fu.make_cifar2_im()
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print('Counter train data: ', Counter(y_train))
print('Counter val data: ', Counter(y_test))

if no_exp == 23:
  print('creating pairs')
  tr_pairs, tr_y = fu.create_pairs_on_set(fu.prep_X(X_train, isResnet), y_train, num_classes)
  ts_pairs, ts_y = fu.create_pairs_on_set(fu.prep_X(X_test, isResnet), y_test, num_classes)
  print(Counter(tr_y))
  print(Counter(ts_y))
  print('init base network')
  base_network = fu.initialize_base_network_resnet(IMAGE_H, IMAGE_W,IMAGE_C)
  # base_network = fu.initialize_base_network(IMAGE_H, IMAGE_W,IMAGE_C)
  # create the left input and point to the base network
  input_shape = (IMAGE_H, IMAGE_W,IMAGE_C)
  input_a = Input(shape=input_shape, name="left_input")
  vect_output_a = base_network(input_a)
  # create the right input and point to the base network
  input_b = Input(shape=input_shape, name="right_input")
  vect_output_b = base_network(input_b)
  # measure the similarity of the two vector outputs
  output = Lambda(fu.euclidean_distance, name="output_layer", output_shape=fu.eucl_dist_output_shape)([vect_output_a, vect_output_b])
  # specify the inputs and output of the model
  sia_model = Model([input_a, input_b], output)

  print('training siamese network...')
  rms = RMSprop()
  sia_model.compile(loss=fu.contrastive_loss_with_margin(margin=1), optimizer=rms)
  history = sia_model.fit([tr_pairs[:,0], tr_pairs[:,1]], tr_y, epochs=epochs_siamese, batch_size=64, validation_data=([ts_pairs[:,0], ts_pairs[:,1]], ts_y), verbose=0)
  print('evaluate')
  loss = sia_model.evaluate(x=[ts_pairs[:,0],ts_pairs[:,1]], y=ts_y)
  # loss = 0.0
  print('predict train')
  y_pred_train = sia_model.predict([tr_pairs[:,0], tr_pairs[:,1]])
  print('compute train acc')
  train_accuracy = fu.compute_accuracy(tr_y, y_pred_train)
  print('predict val')
  y_pred_test = sia_model.predict([ts_pairs[:,0], ts_pairs[:,1]])
  print('compute val acc')
  test_accuracy = fu.compute_accuracy(ts_y, y_pred_test)
  print("Loss = {}, Train Accuracy = {} Test Accuracy = {}".format(loss, train_accuracy, test_accuracy))
  del(ts_pairs)
  del(tr_pairs)

threshold = [0.02, 0.05, 0.1, 0.3, 0.5]
ks = [3,5,7,9]
print('Start exp')
X_test = fu.prep_X(X_test, isResnet)
is_finished = True
while True:
  print(f'start exp no: {no_exp}')
  val_score, test_score = list(), list()
  kfold = StratifiedKFold(num_folds, shuffle=True)
  i_fold = 0
  for train_ix, test_ix in kfold.split(np.zeros(y_train.shape[0]),y_train):
    X_trainsplit, y_trainsplit, X_val_split, y_val_split = X_train[train_ix], y_train[train_ix], X_train[test_ix], y_train[test_ix]
    if no_exp == 0:
      pass
    # elif no_exp == 1:
    elif no_exp >= 28 and no_exp <= 31:
      # 4 exp for 4 different k 3,5,7,9
      k = ks[no_exp-28]
      print("SMOTE with k:",k)
      sm = SMOTE(k_neighbors=k)
      X_trainsplit, y_trainsplit = sm.fit_resample(X_trainsplit.reshape(X_trainsplit.shape[0], -1), y_trainsplit)
      X_trainsplit = X_trainsplit.reshape(X_trainsplit.shape[0], IMAGE_W, IMAGE_H, IMAGE_C)

      if i_fold == num_folds-1:
        if no_exp == 31:
          is_finished = True
        else :
          is_finished = False
          no_exp +=1
    # elif no_exp == 2:
    elif no_exp >= 32 and no_exp <= 35:
      # 4 exp for 4 different k 3,5,7,9
      k = ks[no_exp-32]
      print("ASN smote with k:",k)
      X_trainsplit, y_trainsplit = fu.asn_smote(X_trainsplit.reshape(X_trainsplit.shape[0], -1), y_trainsplit, IMAGE_W, IMAGE_H, IMAGE_C, k=k)
      X_trainsplit = X_trainsplit.reshape(X_trainsplit.shape[0], IMAGE_W, IMAGE_H, IMAGE_C)

      if i_fold == num_folds-1:
        if no_exp == 35:
          is_finished = True
        else :
          is_finished = False
          no_exp +=1
    elif no_exp >= 3 and no_exp <= 7:
      #SIA-SMOTE
      print('SIA SMOTE dis 0.5 with threshold:',threshold[no_exp-3])
      xbase, xsamp, ysamp = fu.SIAMESE_SMOTE_Data(X_trainsplit, y_trainsplit, one_hot = False, maxdist_from_base = 0.5)
      xbase = xbase.reshape(xbase.shape[0],IMAGE_W,IMAGE_H,IMAGE_C)
      xsamp = xsamp.reshape(xsamp.shape[0],IMAGE_W,IMAGE_H,IMAGE_C)
      xbase_prep, xsamp_prep = fu.prep_pixels(xbase, xsamp)
      y_pred = sia_model.predict([xbase_prep, xsamp_prep])
      pred = y_pred.ravel() < threshold[no_exp-3]
      xsamp_sia = xsamp[pred]
      ysamp_sia = ysamp[pred]
      X_trainsplit,y_trainsplit = fu.join_data(X_trainsplit,y_trainsplit,xsamp_sia,ysamp_sia)
      X_trainsplit = X_trainsplit.reshape(-1, IMAGE_W, IMAGE_H, IMAGE_C)
      
      if i_fold == num_folds-1:
        if no_exp == 7:
          is_finished = True
        else :
          is_finished = False
          no_exp +=1
    elif no_exp >= 8 and no_exp <= 12:
      #SIA-SMOTE
      print('SIA SMOTE after SMOTE dist 0.5 with threshold:',threshold[no_exp-8])
      xbase, xsamp, ysamp = fu.SIAMESE_SMOTE_Data(X_trainsplit, y_trainsplit, one_hot = False, maxdist_from_base = 0.5)
      xbase = xbase.reshape(xbase.shape[0],IMAGE_W,IMAGE_H,IMAGE_C)
      xsamp = xsamp.reshape(xsamp.shape[0],IMAGE_W,IMAGE_H,IMAGE_C)
      xbase_prep, xsamp_prep = fu.prep_pixels(xbase, xsamp)
      y_pred = sia_model.predict([xbase_prep, xsamp_prep])
      pred = y_pred.ravel() < threshold[no_exp-8]
      xsamp_sia = xsamp[pred]
      ysamp_sia = ysamp[pred]

      num_over = X_trainsplit[y_trainsplit == 0].shape[0]-xsamp_sia.shape[0]
      if num_over < X_trainsplit[y_trainsplit == 1].shape[0]:
        num_over = X_trainsplit[y_trainsplit == 1].shape[0]
      sm = SMOTE(k_neighbors=5, sampling_strategy={0: X_trainsplit[y_trainsplit == 0].shape[0], 1: num_over})
      X_trainsplit, y_trainsplit = sm.fit_resample(X_trainsplit.reshape(X_trainsplit.shape[0], -1), y_trainsplit)
      X_trainsplit = X_trainsplit.reshape(X_trainsplit.shape[0], IMAGE_W, IMAGE_H, IMAGE_C)

      X_trainsplit,y_trainsplit = fu.join_data(X_trainsplit,y_trainsplit,xsamp_sia,ysamp_sia)
      X_trainsplit = X_trainsplit.reshape(-1, IMAGE_W, IMAGE_H, IMAGE_C)

      if i_fold == num_folds-1:
        if no_exp == 12:
          is_finished = True
        else :
          is_finished = False
          no_exp +=1
    elif no_exp >= 13 and no_exp <= 17:
      #SIA-SMOTE
      print(' Multi SIA SMOTE dist base = 0.5 with threshold:',threshold[no_exp-13])
      eps = 500
      i_multi = 1
      while True:
        xbase, xsamp, ysamp = fu.SIAMESE_SMOTE_Data(X_trainsplit, y_trainsplit, one_hot = False, maxdist_from_base = 0.5)
        xbase = xbase.reshape(xbase.shape[0],IMAGE_W,IMAGE_H,IMAGE_C)
        xsamp = xsamp.reshape(xsamp.shape[0],IMAGE_W,IMAGE_H,IMAGE_C)
        xbase_prep, xsamp_prep = fu.prep_pixels(xbase, xsamp)
        y_pred = sia_model.predict([xbase_prep, xsamp_prep])
        pred = y_pred.ravel() < threshold[no_exp-13]
        if i_multi == 1:
          xsamp_sia = xsamp[pred]
          ysamp_sia = ysamp[pred]
          i_multi+=1
        else:
          xsamp_sia, ysamp_sia = fu.join_data(xsamp_sia,ysamp_sia, xsamp[pred], ysamp[pred])  
        if (X_trainsplit.shape[0]-xsamp_sia.shape[0]) < eps:
          break
        
      X_trainsplit,y_trainsplit = fu.join_data(X_trainsplit,y_trainsplit,xsamp_sia,ysamp_sia)
      X_trainsplit = X_trainsplit.reshape(-1, IMAGE_W, IMAGE_H, IMAGE_C)
      
      if i_fold == num_folds-1:
        if no_exp == 17:
          is_finished = True
        else :
          is_finished = False
          no_exp +=1
    elif no_exp >= 18 and no_exp <= 22:
      #SIA-SMOTE
      print(' Multi SIA SMOTE dist base = 1 with threshold:',threshold[no_exp-18])
      eps = 500
      i_multi = 1
      while True:
        xbase, xsamp, ysamp = fu.SIAMESE_SMOTE_Data(X_trainsplit, y_trainsplit, one_hot = False, maxdist_from_base = 1)
        xbase = xbase.reshape(xbase.shape[0],IMAGE_W,IMAGE_H,IMAGE_C)
        xsamp = xsamp.reshape(xsamp.shape[0],IMAGE_W,IMAGE_H,IMAGE_C)
        xbase_prep, xsamp_prep = fu.prep_pixels(xbase, xsamp)
        y_pred = sia_model.predict([xbase_prep, xsamp_prep])
        pred = y_pred.ravel() < threshold[no_exp-18]
        if i_multi == 1:
          xsamp_sia = xsamp[pred]
          ysamp_sia = ysamp[pred]
          i_multi+=1
        else:
          xsamp_sia, ysamp_sia = fu.join_data(xsamp_sia,ysamp_sia, xsamp[pred], ysamp[pred])  
        if (X_trainsplit.shape[0]-xsamp_sia.shape[0]) < eps:
          break
        
      X_trainsplit,y_trainsplit = fu.join_data(X_trainsplit,y_trainsplit,xsamp_sia,ysamp_sia)
      X_trainsplit = X_trainsplit.reshape(-1, IMAGE_W, IMAGE_H, IMAGE_C)
      
      if i_fold == num_folds-1:
        if no_exp == 22:
          is_finished = True
        else :
          is_finished = False
          no_exp +=1
    elif no_exp >= 23 and no_exp <= 27:
      #SIA-SMOTE
      print('SIA SMOTE after selected SMOTE dist 0.5 with threshold:',threshold[no_exp-23])
      X_trainsplit, y_trainsplit = fu.sia_smote(X_trainsplit.reshape(X_trainsplit.shape[0], -1), y_trainsplit,IMAGE_W,IMAGE_H,IMAGE_C, sia_model, threshold[no_exp-23], one_hot = False, maxdist_from_base = 0.5)
      X_trainsplit = X_trainsplit.reshape(-1, IMAGE_W, IMAGE_H, IMAGE_C)

      if i_fold == num_folds-1:
        if no_exp == 27:
          is_finished = True
        else :
          is_finished = False
          no_exp +=1
    else:
      pass
    
    # prepare pixel data
    # print('normalize')
    X_trainsplit, X_val_split = fu.prep_pixels(X_trainsplit, X_val_split, isResnet)

    # print('define model')
    model = fu.define_model_resnet(IMAGE_H, IMAGE_W,IMAGE_C)
    # model = fu.define_model(IMAGE_H, IMAGE_W,IMAGE_C)
    # fit model
    print('model fitting fold: ',i_fold)
    history = model.fit(X_trainsplit, y_trainsplit, epochs=epochs_classifier, batch_size=64, validation_data=(X_val_split, y_val_split), verbose=0)
    # print('model predict validation_data')
    y_val_pred = model.predict(X_val_split)
    # print('scoring val')
    val_score.append(fu.calculate_score(y_val_split, y_val_pred))
    # print('model predict test data')
    y_test_pred = model.predict(X_test)
    # print('scoring test')
    test_score.append(fu.calculate_score(y_test, y_test_pred))    
    # print('finished fold')
    i_fold=i_fold+1
  print('finish exp no:',no_exp)
  print('printing val score')
  fu.print_score(val_score)
  print('printing test score')
  fu.print_score(test_score)
  if is_finished:
    break
print('Done')
  


loading dataset
170498071/170498071 [==============================] - 13s 0us/step
(5250, 32, 32, 3)
(5250,)
(1050, 32, 32, 3)
(1050,)
Counter train data:  Counter({0: 5000, 1: 250})
Counter val data:  Counter({0: 1000, 1: 50})
creating pairs
Counter({1.0: 498, 0.0: 498})
Counter({1.0: 98, 0.0: 98})
init base network
94765736/94765736 [==============================] - 4s 0us/step
training siamese network...
evaluate
7/7 [==============================] - 0s 20ms/step - loss: 0.9120
predict train
32/32 [==============================] - 2s 16ms/step
compute train acc
predict val
7/7 [==============================] - 0s 16ms/step
compute val acc
Loss = 0.9119957089424133, Train Accuracy = 0.9497991967871486 Test Accuracy = 0.8061224489795918
Start exp
start exp no: 23
SIA SMOTE after selected SMOTE dist 0.5 with threshold: 0.02
108/108 [==============================] - 2s 17ms/step
model fitting fold:  0
33/33 [==============================] - 1s 9ms/step
TN, FP, FN, TP:  991 9 10 4